In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import umap
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
from tqdm import tqdm

In [3]:
from torch import Tensor

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.chdir('../IConNet/')
os.getcwd()

'/home/linh/projects/IConNet'

In [6]:
from einops import rearrange, reduce, repeat

In [7]:
dataset_name = 'ravdess'
experiment_prefix = "scb12"
log_dir = f'../{experiment_prefix}_models/analysis/'
data_dir = "../data/data_preprocessed/"
sr = 16000


In [8]:
from omegaconf import OmegaConf as ocf

dataset_config_path = f'config/dataset/{dataset_name}.yaml'
dataset_config = ocf.load(dataset_config_path)
dataset_config['feature_name'] = 'scb_tfidf'
print(dataset_config)


{'name': 'ravdess', 'dataset_class': 'WaveformDataset', 'root': 'ravdess/', 'audio_dir': 'full_release/', 'feature_dir': 'preprocessing/', 'label_name': 'label_emotion_key', 'feature_name': 'scb_tfidf', 'num_classes': 8, 'label_values': ['neu', 'cal', 'hap', 'sad', 'ang', 'fea', 'dis', 'sur'], 'classnames': ['neu', 'cal', 'hap', 'sad', 'ang', 'fea', 'dis', 'sur'], 'target_labels': ['ang', 'neu', 'sad', 'hap']}


In [9]:
from IConNet.trainer.train_torch import get_dataloader

train_loader, test_loader, batch_size = get_dataloader(
    dataset_config, data_dir, batch_size=16)
print(batch_size)

16


In [10]:
from IConNet.trainer.train_torch import Trainer_SCB10
import torch.optim as optim
from pprint import pprint

class Trainer_custom(Trainer_SCB10):
    def __init__(
            self, 
            batch_size, log_dir, experiment_prefix, device,
            eval_ratio: float=0.2,
            gradient_clip_val: float=0,
            accumulate_grad_batches: int=1,

    ):
        super().__init__(
            batch_size, log_dir, experiment_prefix, device,
            eval_ratio, gradient_clip_val, accumulate_grad_batches)

    def setup(self, model, lr=1e-3):
        super().setup(model=None, lr=lr)
        self.model = model
        self.train_losses_detail = []

    def train_step(self):
        device = self.device
        self.model.train()
        loss = torch.tensor(0., dtype=torch.float64, requires_grad=True)
        for batch_idx, (data, target) in enumerate(self.train_loader):
            data = data.to(device)
            target = target.to(device)
            logits = self.model(data)
            loss = loss + F.cross_entropy(logits.squeeze(), target)
            del data, target
            gc.collect()
            torch.cuda.empty_cache()
            if batch_idx % self.accumulate_grad_batches:
                self.optimizer.zero_grad()
                loss.backward()
                self.clip_gradient()
                self.optimizer.step()
                self.current_step += 1
                loss = loss.item()
                self.train_losses.append(loss)
                if batch_idx % self.val_check_batches == 0: 
                    message = self.gen_log_message(
                        batch_idx, loss, loss_detail=None,
                        memory=None
                    )    
                    self.eval_step(loss=loss, message=message)
                loss = torch.tensor(0., dtype=torch.float64, requires_grad=True)
            self._update_progress_bar()
            
    def fit(self, n_epoch=10, lr=None, test_n_epoch=1):
        device = self.device
        self.total_batches = self.train_batches + self.test_batches
        self.pbar_update = 1/self.total_batches
        if lr is not None:
            self.lr = lr
        self.optimizer = optim.RAdam(self.learnable_parameters, lr=self.lr)
        self.scheduler = optim.lr_scheduler.OneCycleLR(
            self.optimizer, max_lr=0.1,
            steps_per_epoch=self.train_batches, 
            epochs=n_epoch)
        self.model.to(device)
        with tqdm(total=n_epoch) as pbar:
            self.pbar = pbar
            for epoch in range(n_epoch):
                self.current_epoch += 1
                self.train_step()
                self.scheduler.step()
                is_test_epoch = (test_n_epoch is not None and test_n_epoch > 0 and epoch % test_n_epoch == 0)   
                if is_test_epoch:
                    metrics, metrics_details, confusion_matrix = self.test_step()
                    pprint(metrics.compute())
                    pprint(metrics_details.compute())
                    pprint(confusion_matrix.compute())
                    self.save()
                
        self.model.to('cpu')  

In [11]:
class TfidfModel(nn.Module):
    def __init__(self, n_input, seq_num_layer, seq_hidden_dim, cls_dim, num_class=4):
        super().__init__()
        self.lstm_blocks = nn.LSTM(
            input_size=n_input, 
            hidden_size=seq_hidden_dim, 
            num_layers=seq_num_layer, 
            batch_first=True
        )
        self.cls_head = nn.Sequential(
            nn.Linear(seq_hidden_dim, cls_dim, bias=False),
            nn.Linear(cls_dim, cls_dim, bias=False),
            nn.PReLU(cls_dim),
            nn.Linear(cls_dim, num_class, bias=False)
        )
        
    def lstm_forward(self, X: Tensor) -> Tensor:
        X, _ = self.lstm_blocks(X.mT)
        return X.mT

    def forward(self, X: Tensor) -> Tensor:
        X = reduce(self.lstm_forward(X), 'b h n -> b h', 'mean')
        X = self.cls_head(X)
        return X

    def predict(self, X: Tensor) -> tuple[Tensor, Tensor]:
        logits = self.forward(X)
        probs = F.softmax(logits, dim=-1)
        preds = probs.argmax(dim=-1)
        return preds, probs

In [12]:
# trainer = Trainer_custom(batch_size=batch_size, log_dir=log_dir, 
#                   experiment_prefix=experiment_prefix, device=device,
#                  accumulate_grad_batches=2, gradient_clip_val=1.)

# trainer.prepare(train_loader=train_loader, 
#                 test_loader=test_loader, 
#                 batch_size=batch_size)

In [13]:
# model = TfidfModel(
#     n_input=3, 
#     seq_num_layer=2, 
#     seq_hidden_dim=128, 
#     cls_dim=128, 
#     num_class=4)

In [14]:
# trainer.setup(model, lr=1e-4)

In [15]:
# trainer.fit(n_epoch=200, test_n_epoch=10)

  1%|▏                                 | 1.0000000000000007/200 [00:09<39:40, 11.96s/it]

Correct: 46/135 (0.3407)
Saved new best test model: ../scb12_models/analysis/model.epoch=1.step=16.test_acc=0.3407.pt
{'acc_unweighted': tensor(0.2962, device='cuda:0'),
 'acc_weighted': tensor(0.3407, device='cuda:0'),
 'f1s_unweighted': tensor(0.1957, device='cuda:0'),
 'f1s_weighted': tensor(0.2238, device='cuda:0'),
 'rocauc': tensor(0.5680, device='cuda:0'),
 'uar': tensor(0.2962, device='cuda:0'),
 'wap': tensor(0.2516, device='cuda:0')}
{'acc_detail': tensor([0.2105, 0.0000, 0.0000, 0.9744], device='cuda:0'),
 'f1s_detail': tensor([0.3077, 0.0000, 0.0000, 0.4750], device='cuda:0'),
 'precision_detail': tensor([0.5714, 0.0000, 0.0000, 0.3140], device='cuda:0'),
 'recall_detail': tensor([0.2105, 0.0000, 0.0000, 0.9744], device='cuda:0'),
 'rocauc_detail': tensor([0.6343, 0.5436, 0.5343, 0.5598], device='cuda:0')}
tensor([[ 8,  0,  0, 30],
        [ 0,  0,  0, 19],
        [ 5,  0,  0, 34],
        [ 1,  0,  0, 38]], device='cuda:0')


  4%|█▍                                  | 8.18749999999996/200 [01:13<38:48, 12.14s/it]

Correct: 49/135 (0.3630)
Saved new best test model: ../scb12_models/analysis/model.epoch=11.step=176.test_acc=0.3630.pt
{'acc_unweighted': tensor(0.3316, device='cuda:0'),
 'acc_weighted': tensor(0.3630, device='cuda:0'),
 'f1s_unweighted': tensor(0.2707, device='cuda:0'),
 'f1s_weighted': tensor(0.2843, device='cuda:0'),
 'rocauc': tensor(0.6429, device='cuda:0'),
 'uar': tensor(0.3316, device='cuda:0'),
 'wap': tensor(0.2658, device='cuda:0')}
{'acc_detail': tensor([0.6316, 0.1053, 0.0000, 0.5897], device='cuda:0'),
 'f1s_detail': tensor([0.5053, 0.1667, 0.0000, 0.4107], device='cuda:0'),
 'precision_detail': tensor([0.4211, 0.4000, 0.0000, 0.3151], device='cuda:0'),
 'recall_detail': tensor([0.6316, 0.1053, 0.0000, 0.5897], device='cuda:0'),
 'rocauc_detail': tensor([0.7314, 0.7482, 0.5206, 0.5716], device='cuda:0')}
tensor([[24,  0,  0, 14],
        [ 2,  2,  0, 15],
        [16,  2,  0, 21],
        [15,  1,  0, 23]], device='cuda:0')


  8%|██▌                               | 15.375000000000163/200 [02:15<37:23, 12.15s/it]

...

 65%|██████████████████████▏           | 130.37499999998943/200 [19:05<14:16, 12.30s/it]

Correct: 50/135 (0.3704)
{'acc_unweighted': tensor(0.3445, device='cuda:0'),
 'acc_weighted': tensor(0.3704, device='cuda:0'),
 'f1s_unweighted': tensor(0.3395, device='cuda:0'),
 'f1s_weighted': tensor(0.3593, device='cuda:0'),
 'rocauc': tensor(0.6451, device='cuda:0'),
 'uar': tensor(0.3445, device='cuda:0'),
 'wap': tensor(0.3674, device='cuda:0')}
{'acc_detail': tensor([0.5789, 0.1579, 0.3077, 0.3333], device='cuda:0'),
 'f1s_detail': tensor([0.4681, 0.2000, 0.3077, 0.3824], device='cuda:0'),
 'precision_detail': tensor([0.3929, 0.2727, 0.3077, 0.4483], device='cuda:0'),
 'recall_detail': tensor([0.5789, 0.1579, 0.3077, 0.3333], device='cuda:0'),
 'rocauc_detail': tensor([0.6975, 0.7450, 0.5248, 0.6130], device='cuda:0')}
tensor([[22,  1, 13,  2],
        [ 2,  3,  6,  8],
        [17,  4, 12,  6],
        [15,  3,  8, 13]], device='cuda:0')


 69%|████████████████████████           | 137.5624999999927/200 [20:08<12:47, 12.29s/it]

Correct: 51/135 (0.3778)
{'acc_unweighted': tensor(0.3704, device='cuda:0'),
 'acc_weighted': tensor(0.3778, device='cuda:0'),
 'f1s_unweighted': tensor(0.3401, device='cuda:0'),
 'f1s_weighted': tensor(0.3370, device='cuda:0'),
 'rocauc': tensor(0.6572, device='cuda:0'),
 'uar': tensor(0.3704, device='cuda:0'),
 'wap': tensor(0.4100, device='cuda:0')}
{'acc_detail': tensor([0.4737, 0.3158, 0.6410, 0.0513], device='cuda:0'),
 'f1s_detail': tensor([0.5000, 0.3529, 0.4167, 0.0909], device='cuda:0'),
 'precision_detail': tensor([0.5294, 0.4000, 0.3086, 0.4000], device='cuda:0'),
 'recall_detail': tensor([0.4737, 0.3158, 0.6410, 0.0513], device='cuda:0'),
 'rocauc_detail': tensor([0.7271, 0.7663, 0.5150, 0.6205], device='cuda:0')}
tensor([[18,  2, 17,  1],
        [ 0,  6, 13,  0],
        [ 8,  4, 25,  2],
        [ 8,  3, 26,  2]], device='cuda:0')


 72%|█████████████████████████▏         | 143.7499999999955/200 [21:01<08:13,  8.77s/it]


In [23]:
from IConNet.nn.model import M11

In [24]:
model_config_path = 'config/model/m10.yaml'
model_config = ocf.load(model_config_path)
model_config

{'name': 'M10', 'description': 'gated residual FirConv', 'fe': {'n_block': 2, 'n_channel': [64, 64], 'kernel_size': [511, 127], 'stride': [2, 8], 'window_k': [2, 9], 'residual_connection_type': 'concat', 'pooling': 'max'}, 'cls': {'n_block': 2, 'n_hidden_dim': [128, 128]}}

In [25]:
model = M11(config=model_config,
           n_input=3, n_output=4)
model

M11(
  (fe_blocks): FeBlocks(
    (blocks): ModuleList(
      (0-1): 2 x FrontEndBlock(
        (block): ModuleDict(
          (layer): FirConvLayer(
            (conv_layer): ResidualConv1d(
              (pad_layer): PadForConv()
              (stride_layer): DownsampleLayer(
                (pad_right): PadForConv()
              )
              (norm_layer): CustomNormLayer(
                (layer): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
              )
              (act): NLReLU()
            )
          )
          (downsample): ResidualConv1d(
            (pad_layer): PadForConv()
            (stride_layer): DownsampleLayer(
              (pad_right): PadForConv()
            )
            (norm_layer): CustomNormLayer(
              (layer): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
            )
            (act): NLReLU()
          )
        )
      )
    )
  )
  (seq_blocks): Seq2SeqBlocks(
    (blocks): LSTM(64, 64, batch_first=True)
  )
  (cls_hea

In [26]:
trainer = Trainer_custom(batch_size=batch_size, log_dir=log_dir, 
                  experiment_prefix=experiment_prefix, device=device,
                 accumulate_grad_batches=2, gradient_clip_val=1.)

trainer.prepare(train_loader=train_loader, 
                test_loader=test_loader, 
                batch_size=batch_size)

In [27]:
trainer.setup(model, lr=1e-4)

In [28]:
trainer.fit(n_epoch=1000, test_n_epoch=10)

  1%|▏                                 | 1.0000000000000007/200 [00:09<41:31, 12.52s/it]

Correct: 39/135 (0.2889)
Saved new best test model: ../scb12_models/analysis/model.epoch=1.step=24.test_acc=0.2889.pt
{'acc_unweighted': tensor(0.2561, device='cuda:0'),
 'acc_weighted': tensor(0.2889, device='cuda:0'),
 'f1s_unweighted': tensor(0.1410, device='cuda:0'),
 'f1s_weighted': tensor(0.1598, device='cuda:0'),
 'rocauc': tensor(0.4718, device='cuda:0'),
 'uar': tensor(0.2561, device='cuda:0'),
 'wap': tensor(0.2030, device='cuda:0')}
{'acc_detail': tensor([0.9474, 0.0000, 0.0769, 0.0000], device='cuda:0'),
 'f1s_detail': tensor([0.4337, 0.0000, 0.1304, 0.0000], device='cuda:0'),
 'precision_detail': tensor([0.2812, 0.0000, 0.4286, 0.0000], device='cuda:0'),
 'recall_detail': tensor([0.9474, 0.0000, 0.0769, 0.0000], device='cuda:0'),
 'rocauc_detail': tensor([0.4373, 0.4714, 0.5235, 0.4549], device='cuda:0')}
tensor([[36,  0,  2,  0],
        [19,  0,  0,  0],
        [36,  0,  3,  0],
        [37,  0,  2,  0]], device='cuda:0')


  4%|█▍                                  | 8.18749999999996/200 [01:03<38:44, 12.12s/it]

Correct: 50/135 (0.3704)
Saved new best test model: ../scb12_models/analysis/model.epoch=11.step=264.test_acc=0.3704.pt
{'acc_unweighted': tensor(0.3252, device='cuda:0'),
 'acc_weighted': tensor(0.3704, device='cuda:0'),
 'f1s_unweighted': tensor(0.2368, device='cuda:0'),
 'f1s_weighted': tensor(0.2698, device='cuda:0'),
 'rocauc': tensor(0.6294, device='cuda:0'),
 'uar': tensor(0.3252, device='cuda:0'),
 'wap': tensor(0.2122, device='cuda:0')}
{'acc_detail': tensor([0.7368, 0.0000, 0.0000, 0.5641], device='cuda:0'),
 'f1s_detail': tensor([0.5283, 0.0000, 0.0000, 0.4190], device='cuda:0'),
 'precision_detail': tensor([0.4118, 0.0000, 0.0000, 0.3333], device='cuda:0'),
 'recall_detail': tensor([0.7368, 0.0000, 0.0000, 0.5641], device='cuda:0'),
 'rocauc_detail': tensor([0.7244, 0.6919, 0.5013, 0.5999], device='cuda:0')}
tensor([[28,  0,  1,  9],
        [ 4,  0,  0, 15],
        [19,  0,  0, 20],
        [17,  0,  0, 22]], device='cuda:0')


  8%|██▌                               | 15.375000000000163/200 [01:58<37:45, 12.27s/it]

...

 65%|██████████████████████▏           | 130.37499999998943/200 [16:32<14:00, 12.08s/it]

Correct: 59/135 (0.4370)
{'acc_unweighted': tensor(0.4501, device='cuda:0'),
 'acc_weighted': tensor(0.4370, device='cuda:0'),
 'f1s_unweighted': tensor(0.3880, device='cuda:0'),
 'f1s_weighted': tensor(0.3737, device='cuda:0'),
 'rocauc': tensor(0.7058, device='cuda:0'),
 'uar': tensor(0.4501, device='cuda:0'),
 'wap': tensor(0.4543, device='cuda:0')}
{'acc_detail': tensor([0.6842, 0.5263, 0.0256, 0.5641], device='cuda:0'),
 'f1s_detail': tensor([0.5591, 0.4762, 0.0488, 0.4681], device='cuda:0'),
 'precision_detail': tensor([0.4727, 0.4348, 0.5000, 0.4000], device='cuda:0'),
 'recall_detail': tensor([0.6842, 0.5263, 0.0256, 0.5641], device='cuda:0'),
 'rocauc_detail': tensor([0.7434, 0.8099, 0.6183, 0.6514], device='cuda:0')}
tensor([[26,  1,  0, 11],
        [ 1, 10,  1,  7],
        [17,  6,  1, 15],
        [11,  6,  0, 22]], device='cuda:0')


 69%|████████████████████████           | 137.5624999999927/200 [17:26<12:40, 12.19s/it]

Correct: 60/135 (0.4444)
{'acc_unweighted': tensor(0.3959, device='cuda:0'),
 'acc_weighted': tensor(0.4444, device='cuda:0'),
 'f1s_unweighted': tensor(0.3705, device='cuda:0'),
 'f1s_weighted': tensor(0.4114, device='cuda:0'),
 'rocauc': tensor(0.6897, device='cuda:0'),
 'uar': tensor(0.3959, device='cuda:0'),
 'wap': tensor(0.4153, device='cuda:0')}
{'acc_detail': tensor([0.7105, 0.0526, 0.2821, 0.5385], device='cuda:0'),
 'f1s_detail': tensor([0.5934, 0.0833, 0.3438, 0.4615], device='cuda:0'),
 'precision_detail': tensor([0.5094, 0.2000, 0.4400, 0.4038], device='cuda:0'),
 'recall_detail': tensor([0.7105, 0.0526, 0.2821, 0.5385], device='cuda:0'),
 'rocauc_detail': tensor([0.7371, 0.7595, 0.5951, 0.6669], device='cuda:0')}
tensor([[27,  0,  2,  9],
        [ 3,  1,  6,  9],
        [13,  2, 11, 13],
        [10,  2,  6, 21]], device='cuda:0')


 72%|█████████████████████████▏         | 143.7499999999955/200 [18:12<07:07,  7.60s/it]
